# Exercice 2

## Question 1

On souhaite une configuration où le predicteur de Bayes est différent selon l'utilisation de la square loss ou de l'absolute loss.
On a vu que le prédicteur de Bayes dans le cas de la square loss est l'espérence conditionnelle de Y sachant X.
Dans le cas de l'absolute loss, c'est la médiane de Y sachant X.

On veut donc prendre une loi de probabilité selon laquelle la médiane est différente de l'espérence.

Soit la modelisation suivante : <br>
X suit une loi uniforme sur [0.1; 0.9]. <br>
Y suit une loi géomètrique de paramètre(X).

Pour la square loss: 
$$
\begin{equation}
\begin{split}
f^*(x) & = E[Y|X] \\
& = 1/X 
\end{split}
\end{equation}
$$
$$
\begin{equation}
\begin{split}
R(f^*(x)) & = E_{X, Y}[l(y, f^*(x))] \\
& = E_X[E_Y[l(y, f^*(x)) | X = x]] \\
& = E_X[E_Y[(y - \frac{1}{x})^2 | X = x]] \\
& = E_X[\sum_{y=1}^{+\infty} (y - \frac{1}{x})^2 (1-x)^{y-1}x] \\
& = \int_{0.1}^{0.9}(\sum_{y=1}^{+\infty} (y - \frac{1}{x})^2 (1-x)^{y-1}x)\frac{1}{0.9 - 0.1}dx \\
\end{split}
\end{equation}


En utilisant Wolframe alpha on calcule le Risk De bayes

![Bayes Risk for square loss](./images/squareloss_bayes_risk.png)

Pour la absolute loss :
$$
\begin{equation}
\begin{split}
f^*(x) & = Mediane Y|X \\
& = \lceil{\frac{-log(2)}{log(1-X)}} \rceil
\end{split}
\end{equation}
$$
$$
\begin{equation}
\begin{split}
R(f^*(x)) & = E_{X, Y}[l(y, f^*(x))] \\
& = E_X[E_Y[l(y, f^*(x)) | X = x]] \\
& = E_X[E_Y[|y - \lceil{\frac{-log(2)}{log(1-X)}} \rceil| | X = x]] \\
& = E_X[\sum_{y=1}^{+\infty} |y - \lceil{\frac{-log(2)}{log(1-X)}} \rceil| P(Y = y | X = x)] \\
& = E_X[\sum_{y=1}^{\lfloor(\frac{1}{x})} (\lceil{\frac{-log(2)}{log(1-X)}} \rceil - y) P(Y = y | X = x)
+
\sum_{y=\lfloor \frac{1}{x} \rfloor}^{+\infty} (y - \lceil{\frac{-log(2)}{log(1-X)}} \rceil) P(Y = y | X = x)
] \\
& = \int_{0.1}^{0.9}(\sum_{y=1}^{\lfloor(\frac{1}{x})} (\lceil{\frac{-log(2)}{log(1-X)}} \rceil - y) (1-x)^{y-1}x
+
\sum_{y=\lfloor \frac{1}{x} \rfloor}^{+\infty} (y - \lceil{\frac{-log(2)}{log(1-X)}} \rceil) (1-x)^{y-1}x)\frac{1}{0.9 - 0.1}dx \\
\end{split}
\end{equation}

Nous essayons de calculer avec Wolframe alpha mais le calcul étant trop complexe nous ne parvenons pas à le calculer avec la version gratuite de Wolframe Alpha

![Bayes Risk with the absolute loss and the bayes predictor](images/absoluteloss_with_median_bayes_risk.png)

Cependant, étrangement le calcul se fait lorsque l'on utilise l'esperance Y sachant X, c'est à dire ici $ \frac{1}{x} $ comme prédicteur. On vérifiera alors ce calcul expérimentalement.

![Bayes Risk with absolute loss and E[Y|X] predictor](images/absoluteloss_with_expected_bayes_predictor.png)

### Code

In [1]:
import numpy as np

rng = np.random.default_rng()

In [2]:
# define the loss functions
def square_loss(x , y) -> float:
    return (x - y)**2

def absolute_loss(x , y) -> float:
    return np.abs(x - y)

In [3]:
# generate the data
from typing import Tuple


def generate_data(n_samples: int) -> Tuple[np.array, np.array]:
    """
    n_samples: number of samples to generate
    return: X, Y
    """
    X = rng.uniform(0.1, 0.9, size=n_samples) # we restrict the range to avoid 0 and 1 and to explode the loss
    Y = rng.geometric(X) # Y follows a geometric distribution with parameter X
    
    return X, Y

In [4]:
# define the bayes predictor for the square loss
# which is the conditional expectation
def square_bayes_predictor(X: np.array) -> np.array:
    """
    It generates the prediction for the bayes predictor which is the conditional expectation.
    Here it's 1/X because Y follows geometric distribution with parameter X.
    """
    return 1/X

In [5]:
# define the bayes predictor for the absolute loss
# which is the median

def absolute_bayes_predictor(X: np.array) -> np.array:
    """
    It generates the prediction for the bayes predictor which is the median.
    Here it's supp(-ln(2) / ln(q))) where q = 1 - X.
    https://fr.wikipedia.org/wiki/Loi_g%C3%A9om%C3%A9trique
    """
    return np.ceil((- np.log(2)) / np.log(1 - X))

In [6]:
# define a bad estimator to compare results
def bad_estimator(X: np.array) -> np.array:
    """
    X: np.array of inputs
    Given a set of inputs, return a bad estimator
    """

    return rng.geometric(0.5, size=len(X)) # random estimator

In [7]:
# define the empirical risk function
def empirical_risk(estimator, loss, X: np.array, Y: np.array) -> float:
    """
    estimator: function that takes X and returns Y
    loss: function that takes two values and returns a loss
    X: np.array of inputs
    Y: np.array of outputs
    Given a set of inputs and outputs, return the empirical risk
    """

    Y_pred = estimator(X)
    l = loss(Y, Y_pred)
    return l.mean()

### Run the simulation

In [8]:
X, Y = generate_data(int(1e6))
print('X = ', X)
print("Y = ", Y)
print("")

print("with the square loss")
print("Empirical risk for the bad estimator: ", empirical_risk(bad_estimator, square_loss, X, Y))
print("Empirical risk for the squared bayes predictor: ", empirical_risk(square_bayes_predictor, square_loss, X, Y))
print("Empirical risk for the absolute bayes predictor: ", empirical_risk(absolute_bayes_predictor, square_loss, X, Y))
print("")
print("with the absolute loss")
print("Empirical risk for the bad estimator: ", empirical_risk(bad_estimator, absolute_loss, X, Y))
print("Empirical risk for the squared bayes predictor: ", empirical_risk(square_bayes_predictor, absolute_loss, X, Y))
print("Empirical risk for the absolute bayes predictor: ", empirical_risk(absolute_bayes_predictor, absolute_loss, X, Y))


X =  [0.51081115 0.7662838  0.35429327 ... 0.11382151 0.12447091 0.10446652]
Y =  [1 1 2 ... 9 1 5]

with the square loss
Empirical risk for the bad estimator:  14.712686
Empirical risk for the squared bayes predictor:  8.497643756298206
Empirical risk for the absolute bayes predictor:  9.460232

with the absolute loss
Empirical risk for the bad estimator:  2.021086
Empirical risk for the squared bayes predictor:  1.585885175370916
Empirical risk for the absolute bayes predictor:  1.438746


Pour la square loss, on a bien les deux estimateur de bayes qui sont meilleurs que l'estimateur aléatoire. 
<br>
<br>
De plus, comme prévu le squared bayes predictor a un risque empirique plus faible que l'absolute bayes predictor confirmant qu'il est meilleur lorsque l'on utilise la squared loss. <br>
Enfin, 8.49 est proche de la valeur théorique 8.36 que l'on cherchait à estimer

Pour la absolute loss, on retrouve les mêmes conclusions attendues. L'absolute bayes predictor a un risque empirique plus faible que le squared bayes predictor confirmant qu'il est meilleur lorsque l'on considère l'absolute loss.
On retrouve également le résultat théorique attendu, à savoir 1.58 contre 1.57 pour le risque empirique du prédicteur E[Y|X] avec l'absolute loss.



## Question 2

$
\begin{equation}
\begin{split}
 f^*(x) & = arg min E[|y − z||X = x] \\
 E[|y − z||X = x] & = \int_{-\infty}^{+\infty} |y - z| p_{Y|X=x}dy \\
 & = \int_{-\infty}^{z} (z - y)p_{Y|X=x}dy + \int_{z}^{+\infty} (y - z)p_{Y|X=x}dy \\
 & = \int_{-\infty}^{z} -(y - z)p_{Y|X=x}dy + \int_{z}^{+\infty} (y - z)p_{Y|X=x}dy \\ 
 \end{split}
 \end{equation}
$
Pour trouver le minimum, on va chercher quand est-ce que la dérivée s'annule.

$$
\begin{equation}
\begin{split}
\frac{\partial E[|y − z||X = x]}{\partial z} & = \int_{-\infty}^{z} p_{Y|X=x}dy + \int_{z}^{+\infty} -p_{Y|X=x}dy = 0 \\
\int_{-\infty}^{z} p_{Y|X=x}dy & = \int_{z}^{+\infty} -p_{Y|X=x}dy \\
\implies 2 * \int_{-\infty}^{z} p_{Y|X=x}dy & = \int_{-\infty}^{+\infty} p_{Y|X=x}dy = 1 \\
\int_{-\infty}^{z} p_{Y|X=x}dy & = \frac{1}{2} \\
\text{Donc } F_{Y|X}(z) = \frac{1}{2} \\ \\
\end{split}
\end{equation}
$$
Vérifions que c'est bien un minimum
$$
\begin{equation}
\begin{split}
\frac{\partial E[|y − z||X = x]}{\partial z}  = \int_{-\infty}^{z} p_{Y|X=x}dy - \int_{z}^{+\infty} p_{Y|X=x}dy & < 0 \\
F_{Y|X}(z) - (1 - F_{Y|X}(z)) < 0 \\
2*F_{Y|X}(z) < 1 \\
F_{Y|X}(z) < \frac{1}{2} \\
\end{split}
\end{equation}
$$
F_{Y|X} est une fonction de répartition donc elle est croissante sur l'intervalle [0, 1]. On peut donc tracer le tableau de variation suivant.

![Tableau de variation](images/tableau_variation.jpeg)

$
\text{Donc } z = f^{*}(x) \text{ est la médiane de Y sachant X}
$